In [1]:
#Importing all libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load datasets
customers = pd.read_csv(r'C:\Users\A.Rohith Venkatesh\Downloads\Customers.csv')
transactions = pd.read_csv(r'C:\Users\A.Rohith Venkatesh\Downloads\Transactions.csv')

# Data preprocessing
transactions_summary = transactions.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'ProductID': lambda x: ' '.join(x.astype(str))
}).reset_index()

customer_profiles = pd.merge(customers, transactions_summary, on='CustomerID', how='left')
customer_profiles.fillna(0, inplace=True)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure customer_profiles is defined
# Feature selection for clustering
clustering_data = customer_profiles.drop(columns=['CustomerID', 'CustomerName', 'ProductID'])

# Initialize and Standardize features
scaler = StandardScaler()  # Initialize the scaler
scaled_clustering_data = scaler.fit_transform(clustering_data)

# Determine optimal number of clusters
db_scores = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(scaled_clustering_data)
    db_score = davies_bouldin_score(scaled_clustering_data, clusters)
    db_scores.append(db_score)

# Visualize DB Index scores
plt.figure(figsize=(10, 6))
sns.lineplot(x=range(2, 11), y=db_scores, marker='o')
plt.title('Davies-Bouldin Index for Different Cluster Counts')
plt.xlabel('Number of Clusters')
plt.ylabel('DB Index')
plt.show()

# Final Clustering with Optimal K
optimal_k = db_scores.index(min(db_scores)) + 2
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42)
customer_profiles['Cluster'] = final_kmeans.fit_predict(scaled_clustering_data)

# Save results
customer_profiles.to_csv('Clustering_Results.csv', index=False)

# Visualize Clusters using PCA
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(scaled_clustering_data)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=reduced_data[:, 0], y=reduced_data[:, 1], hue=customer_profiles['Cluster'], palette='viridis')
plt.title('Customer Clusters')
plt.show()

ValueError: could not convert string to float: 'South America'